# 関数定義

In [36]:
import copy

def argMax(V, R, x, y,cost=1, miss=None):
    xmax = len(V[0])-1
    ymax = len(V)-1
    Vster = []
    Varror = []
    correct = 1.0 
    
    if V[y][x]==0:
        return 0,"・"
    
    if x>0 and V[y][x-1] is not None:
        Vster.append(V[y][x-1] - cost - R[y][x-1]) 
        Varror.append("←")
        correct -= 0.1
    if x<xmax and V[y][x+1] is not None:
        Vster.append(V[y][x+1] - cost - R[y][x+1])
        Varror.append("→")
        correct -= 0.1
    if y>0 and V[y-1][x] is not None:
        Vster.append(V[y-1][x] - cost - R[y-1][x])
        Varror.append("↑")
        correct -= 0.1
    if y<ymax and V[y+1][x] is not None:
        Vster.append(V[y+1][x] - cost - R[y+1][x])
        Varror.append("↓")
        correct -= 0.1
            
    if V[y][x] is None:
        return None,"　"
    
    arror = Varror[Vster.index(max(Vster))]
    
    if miss is None:
        return max(Vster), arror
    else:
        maxValue = max(Vster)
        
        totalValue = (maxValue) * (correct + miss)
        Vster.remove(maxValue)
        
        for i in Vster:
            totalValue += i * miss
            
        Vster.append(maxValue)
        
        return totalValue, arror
    
def showValueMap(V, header = None):
    if header is not None:
        print header
        
    for x in V:
        if type(V[0][0])==float:
            print "[",
            for i in x:
                if i==None:
                    print str(i).rjust(7),
                    print ',',
                else :
                    print '{:7.2f}'.format(i),
                    print ',',
            print "]"
            
        elif type(V[0][0])==int:
            print "[",
            for i in x:
                print "{0:4d}".format(i),
            print "]"
            
        else:
            print str(x).decode("string-escape")
    
    print "----------------------"

def checkConnivent(list1, list2, value):
    checkList = []
    
    for y in range(len(list1)):
            for x in range(len(list1[0])):
                if list1[y][x] == None:
                    checkList.append(True)
                else:
                    checkList.append(abs(list1[y][x] - list2[y][x]) <= value)
                
    return all(elem == True for elem in checkList)
    

def kachihanpuku(V, R, miss=None, connError = 1):
    Vpre = copy.deepcopy(V)
    Vster =  [[0 for i in range(len(V[0]))] for j in range(len(V))]
    Varror = [[0 for i in range(len(V[0]))] for j in range(len(V))]
    i = 1
   
    #for i in range(5):
    while True:
        for y in range(len(V)):
            for x in range(len(V[y])):
                Vster[y][x],Varror[y][x] = argMax(Vpre, R, x, y, miss=miss)
        
        showValueMap(Vster, "sweep "+str(i))
        i+=1
        
        #if Vpre == Vster:
        if checkConnivent(Vpre,Vster,connError):
            break
        Vpre = copy.deepcopy(Vster)
    return Vster, Varror

# 問題1
<img src="https://lab.ueda.asia/wp-content/uploads/2016/11/questions1.png">

In [37]:
V=[[-10, -10,  0],
   [-10, -10, -10],
   [-10, -10, -10]]

R=[[0, 0, 0],
   [0, 0, 0],
   [0, 0, 0]]

ValueMap, ArrorMap = kachihanpuku(V, R)

print "\n------solution ------\n"
showValueMap(ValueMap , "---Value Map---")
showValueMap(ArrorMap, "---Arror Map---")

sweep 1
[  -11   -1    0 ]
[  -11  -11   -1 ]
[  -11  -11  -11 ]
----------------------
sweep 2
[   -2   -1    0 ]
[  -12   -2   -1 ]
[  -12  -12   -2 ]
----------------------
sweep 3
[   -2   -1    0 ]
[   -3   -2   -1 ]
[  -13   -3   -2 ]
----------------------
sweep 4
[   -2   -1    0 ]
[   -3   -2   -1 ]
[   -4   -3   -2 ]
----------------------
sweep 5
[   -2   -1    0 ]
[   -3   -2   -1 ]
[   -4   -3   -2 ]
----------------------

------solution ------

---Value Map---
[   -2   -1    0 ]
[   -3   -2   -1 ]
[   -4   -3   -2 ]
----------------------
---Arror Map---
['→', '→', '・']
['→', '→', '↑']
['→', '→', '↑']
----------------------


# 問題2

In [38]:
V=[[-10, -10,  0],
   [-10, -10, -10],
   [-10, -10, -10]]

R1=[[0, 1, 0],
   [0, 0, 0],
   [0, 0, 0]]

R2=[[0, 10, 0],
   [0, 0, 0],
   [0, 0, 0]]

print "水たまりのペナルティが1の場合"
ValueMap,ArrorMap = kachihanpuku(V, R1)
print "\n------solution ------\n"
showValueMap(ValueMap , "---Value Map---")
showValueMap(ArrorMap, "---Arror Map---")

print "\n"
print "水たまりのペナルティが10の場合"
ValueMap,ArrorMap = kachihanpuku(V, R2)
print "\n------solution ------\n"
showValueMap(ValueMap , "---Value Map---")
showValueMap(ArrorMap, "---Arror Map---")

水たまりのペナルティが1の場合
sweep 1
[  -11   -1    0 ]
[  -11  -11   -1 ]
[  -11  -11  -11 ]
----------------------
sweep 2
[   -3   -1    0 ]
[  -12   -2   -1 ]
[  -12  -12   -2 ]
----------------------
sweep 3
[   -3   -1    0 ]
[   -3   -2   -1 ]
[  -13   -3   -2 ]
----------------------
sweep 4
[   -3   -1    0 ]
[   -3   -2   -1 ]
[   -4   -3   -2 ]
----------------------
sweep 5
[   -3   -1    0 ]
[   -3   -2   -1 ]
[   -4   -3   -2 ]
----------------------

------solution ------

---Value Map---
[   -3   -1    0 ]
[   -3   -2   -1 ]
[   -4   -3   -2 ]
----------------------
---Arror Map---
['→', '→', '・']
['→', '→', '↑']
['→', '→', '↑']
----------------------


水たまりのペナルティが10の場合
sweep 1
[  -11   -1    0 ]
[  -11  -11   -1 ]
[  -11  -11  -11 ]
----------------------
sweep 2
[  -12   -1    0 ]
[  -12   -2   -1 ]
[  -12  -12   -2 ]
----------------------
sweep 3
[  -12   -1    0 ]
[   -3   -2   -1 ]
[  -13   -3   -2 ]
----------------------
sweep 4
[   -4   -1    0 ]
[   -3   -2   -1 ]
[   -4  

# 問題3

In [30]:
V=[[-100, -100,    0],
   [-100, -100, -100],
   [-100, -100, -100]]

R=[[0, 0, 0],
   [0, 0, 0],
   [0, 0, 0]]

miss=0.1

ValueMap,ArrorMap = kachihanpuku(V, R1, miss, 1)
print "\n------solution ------\n"
showValueMap(ValueMap , "---Value Map---")
showValueMap(ArrorMap, "---Arror Map---")

sweep 1
[ -101.10 ,  -21.00 ,    0.00 , ]
[ -101.00 , -101.10 ,  -21.00 , ]
[ -101.00 , -101.00 , -101.00 , ]
----------------------
sweep 2
[  -30.90 ,  -21.22 ,    0.00 , ]
[ -102.02 ,  -38.10 ,  -21.21 , ]
[ -102.00 , -102.01 ,  -30.00 , ]
----------------------
sweep 3
[  -31.20 ,   -7.90 ,    0.00 , ]
[  -39.73 ,  -38.47 ,   -7.81 , ]
[ -103.01 ,  -39.01 ,  -30.29 , ]
----------------------
sweep 4
[  -12.98 ,   -7.97 ,    0.00 , ]
[  -40.11 ,  -15.23 ,   -7.88 , ]
[  -40.08 ,  -39.38 ,  -11.93 , ]
----------------------
sweep 5
[  -13.08 ,   -3.82 ,    0.00 , ]
[  -16.92 ,  -15.36 ,   -3.72 , ]
[  -40.45 ,  -16.08 ,  -12.03 , ]
----------------------
sweep 6
[   -7.03 ,   -3.84 ,    0.00 , ]
[  -17.05 ,   -7.38 ,   -3.74 , ]
[  -17.16 ,  -16.20 ,   -5.95 , ]
----------------------
sweep 7
[   -7.06 ,   -2.44 ,    0.00 , ]
[   -9.08 ,   -7.43 ,   -2.33 , ]
[  -17.29 ,   -8.22 ,   -5.98 , ]
----------------------
sweep 8
[   -5.01 ,   -2.45 ,    0.00 , ]
[   -9.12 ,   -4.71 ,   -2.

# 問題4

In [39]:
V=[[-100, -100,    0],
   [-100, -100, -100],
   [-100, -100, -100]]

R1=[[0, 1, 0],
   [0, 0, 0],
   [0, 0, 0]]

R2=[[0, 10, 0],
   [0, 0, 0],
   [0, 0, 0]]

R3=[[0, 100, 0],
   [0, 0, 0],
   [0, 0, 0]]

miss = 0.1

print "水たまりのペナルティが1の場合"
ValueMap,ArrorMap = kachihanpuku(V, R1, miss, 1)
print "\n------solution ------\n"
showValueMap(ValueMap , "---Value Map---")
showValueMap(ArrorMap, "---Arror Map---")


print "\n"
print "水たまりのペナルティが10の場合"
ValueMap,ArrorMap = kachihanpuku(V, R2, miss, 1)
print "\n------solution ------\n"
showValueMap(ValueMap , "---Value Map---")
showValueMap(ArrorMap, "---Arror Map---")

print "\n"
print "水たまりのペナルティが100の場合"
ValueMap,ArrorMap = kachihanpuku(V, R3, miss, 1)
print "\n------solution ------\n"
showValueMap(ValueMap , "---Value Map---")
showValueMap(ArrorMap, "---Arror Map---")

水たまりのペナルティが1の場合
sweep 1
[ -101.10 ,  -21.00 ,    0.00 , ]
[ -101.00 , -101.10 ,  -21.00 , ]
[ -101.00 , -101.00 , -101.00 , ]
----------------------
sweep 2
[  -30.90 ,  -21.22 ,    0.00 , ]
[ -102.02 ,  -38.10 ,  -21.21 , ]
[ -102.00 , -102.01 ,  -30.00 , ]
----------------------
sweep 3
[  -31.20 ,   -7.90 ,    0.00 , ]
[  -39.73 ,  -38.47 ,   -7.81 , ]
[ -103.01 ,  -39.01 ,  -30.29 , ]
----------------------
sweep 4
[  -12.98 ,   -7.97 ,    0.00 , ]
[  -40.11 ,  -15.23 ,   -7.88 , ]
[  -40.08 ,  -39.38 ,  -11.93 , ]
----------------------
sweep 5
[  -13.08 ,   -3.82 ,    0.00 , ]
[  -16.92 ,  -15.36 ,   -3.72 , ]
[  -40.45 ,  -16.08 ,  -12.03 , ]
----------------------
sweep 6
[   -7.03 ,   -3.84 ,    0.00 , ]
[  -17.05 ,   -7.38 ,   -3.74 , ]
[  -17.16 ,  -16.20 ,   -5.95 , ]
----------------------
sweep 7
[   -7.06 ,   -2.44 ,    0.00 , ]
[   -9.08 ,   -7.43 ,   -2.33 , ]
[  -17.29 ,   -8.22 ,   -5.98 , ]
----------------------
sweep 8
[   -5.01 ,   -2.45 ,    0.00 , ]
[   -9.12 ,

# 問題5

In [32]:
V=[[-100, -100,     0],
   [-100, -100, -100],
   [-100, -100, -100],
   [ None,  None,     0]]

R1=[[0, 1, 0],
   [0, 0, 0],
   [0, 0, 0],
   [None, None, 0]]

R2=[[0, 10, 0],
   [0, 0, 0],
   [0, 0, 0],
   [None, None, 0]]

R3=[[0, 100, 0],
   [0, 0, 0],
   [0, 0, 0],
   [None, None, 0]]

miss = 0.1

print "水たまりのペナルティが1の場合"
ValueMap,ArrorMap = kachihanpuku(V, R1, miss, 1)
print "\n------solution ------\n"
showValueMap(ValueMap , "---Value Map---")
showValueMap(ArrorMap, "---Arror Map---")


print "\n"
print "水たまりのペナルティが10の場合"
ValueMap,ArrorMap = kachihanpuku(V, R2, miss, 1)
print "\n------solution ------\n"
showValueMap(ValueMap , "---Value Map---")
showValueMap(ArrorMap, "---Arror Map---")

print "\n"
print "水たまりのペナルティが100の場合"
ValueMap,ArrorMap = kachihanpuku(V, R3, miss, 1)
print "\n------solution ------\n"
showValueMap(ValueMap , "---Value Map---")
showValueMap(ArrorMap, "---Arror Map---")

水たまりのペナルティが1の場合
sweep 1
[ -101.10 ,  -21.00 ,    0.00 , ]
[ -101.00 , -101.10 ,  -21.00 , ]
[ -101.00 , -101.00 ,  -21.00 , ]
[    None ,    None ,    0.00 , ]
----------------------
sweep 2
[  -30.90 ,  -21.22 ,    0.00 , ]
[ -102.02 ,  -38.10 ,  -13.21 , ]
[ -102.00 ,  -38.01 ,  -13.20 , ]
[    None ,    None ,    0.00 , ]
----------------------
sweep 3
[  -31.20 ,   -7.90 ,    0.00 , ]
[  -39.73 ,  -26.47 ,   -6.13 , ]
[  -45.41 ,  -25.57 ,   -6.12 , ]
[    None ,    None ,    0.00 , ]
----------------------
sweep 4
[  -12.98 ,   -6.77 ,    0.00 , ]
[  -29.84 ,  -12.71 ,   -4.26 , ]
[  -27.99 ,  -13.09 ,   -4.17 , ]
[    None ,    None ,    0.00 , ]
----------------------
sweep 5
[  -10.97 ,   -3.57 ,    0.00 , ]
[  -15.27 ,   -9.05 ,   -2.69 , ]
[  -15.76 ,   -8.41 ,   -2.73 , ]
[    None ,    None ,    0.00 , ]
----------------------
sweep 6
[   -6.64 ,   -3.00 ,    0.00 , ]
[  -10.91 ,   -5.71 ,   -2.18 , ]
[  -10.09 ,   -5.67 ,   -2.11 , ]
[    None ,    None ,    0.00 , ]
-----